In [2]:
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import matplotlib.pyplot
import matplotlib.pyplot as plt
from numpy import where
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC

In [3]:
data_project = pd.read_csv('../datasets/missingmarks_clean_data.csv')

In [26]:
data_project.describe().T

,count,mean,std,min,25%,50%,75%,max
institution,10047.0,10607.570319,12.004388,10592.0,10597.0,10607.0,10617.0,10627.0
system,10047.0,1.227630,0.419323,1.0,1.0,1.0,1.0,2.0
data_loss_gen,10047.0,1.781925,0.412959,1.0,2.0,2.0,2.0,2.0
data_loss_human,10047.0,1.977904,0.668173,1.0,2.0,2.0,2.0,3.0
data_loss_machine,10047.0,1.832288,0.766250,1.0,1.0,2.0,2.0,4.0
data_loss_hybrid,10047.0,1.764905,0.424079,1.0,2.0,2.0,2.0,2.0
system_error_chance,10047.0,1.387379,0.487176,1.0,1.0,1.0,2.0,2.0
feature,10047.0,1.161740,0.368230,1.0,1.0,1.0,1.0,2.0


In [4]:
data_project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10047 entries, 0 to 10046
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   institution          10047 non-null  int64
 1   system               10047 non-null  int64
 2   data_loss_gen        10047 non-null  int64
 3   data_loss_human      10047 non-null  int64
 4   data_loss_machine    10047 non-null  int64
 5   data_loss_hybrid     10047 non-null  int64
 6   system_error_chance  10047 non-null  int64
 7   feature              10047 non-null  int64
dtypes: int64(8)
memory usage: 628.1 KB


In [5]:
x = data_project.drop(columns=['institution', 'system', 'feature'])
y= data_project['feature']


In [21]:
x.isna().sum()

data_loss_gen          0
data_loss_human        0
data_loss_machine      0
data_loss_hybrid       0
system_error_chance    0
dtype: int64

In [22]:
x.isnull().sum()

data_loss_gen          0
data_loss_human        0
data_loss_machine      0
data_loss_hybrid       0
system_error_chance    0
dtype: int64

In [23]:
y.isna().sum()

0

In [24]:
y.isnull().sum()

0

In [6]:
counter = Counter(y)
print(counter)

Counter({1: 8422, 2: 1625})


In [7]:
model_1_svm  = SVC(gamma='scale')

In [8]:
cross_val = RepeatedStratifiedKFold(
    n_splits=10,
    n_repeats=3,
    random_state=1
)

In [10]:
perf_score = cross_val_score(model_1_svm, x, y, scoring='roc_auc', cv=cross_val, n_jobs=-1)

In [11]:
print('Mean ROC AUC: %.3f' % mean(perf_score))

Mean ROC AUC: 0.924


In [12]:
model_2_svm = SVC(gamma='scale', class_weight='balanced')

In [13]:
cross_val = RepeatedStratifiedKFold(
    n_splits=10,
    n_repeats=3,
    random_state=1
)

In [14]:
perf_score_2 = cross_val_score(model_2_svm, x, y, scoring='roc_auc', cv=cross_val, n_jobs=-1
                               )

In [15]:
print('Mean ROC AUC %.3f' % mean(perf_score_2))

Mean ROC AUC 0.920


In [16]:
model_3_svm = SVC(gamma='scale')

In [17]:
bal = [{0:100, 1:1}, {0:10, 1:1}, {0:1, 1:1}, {0:1, 1:10}, {0:1, 1:100}]
parameter_grid = dict(class_weight=bal)

In [18]:
cross_val = RepeatedStratifiedKFold(
    n_splits=10, 
    n_repeats=3,
    random_state=1
)

In [19]:
grid = GridSearchCV(estimator=model_3_svm, param_grid=parameter_grid, n_jobs=-1, cv=cross_val, scoring='roc_auc')

In [20]:
grid_out = grid.fit(x, y)

C:\Users\Wessware\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


ValueError: Class label 0 not present.